In [20]:
from langchain_groq  import ChatGroq

In [21]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key =os.getenv("GROQ_API_KEY")


In [22]:
llm = ChatGroq(
    temperature=0,
    groq_api_key=api_key,
    model_name='llama-3.1-70b-versatile'
)

In [23]:
response=llm.invoke("The first person to land on moon was...")
print(response)

content="Neil Armstrong.  He stepped onto the moon's surface on July 20, 1969, during the Apollo 11 mission." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 44, 'total_tokens': 72, 'completion_time': 0.112, 'prompt_time': 0.012206236, 'queue_time': 0.004979612999999999, 'total_time': 0.124206236}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_9260b4bb2e', 'finish_reason': 'stop', 'logprobs': None} id='run-da4ba36b-aa66-469a-9e3b-5c58a5e6106e-0' usage_metadata={'input_tokens': 44, 'output_tokens': 28, 'total_tokens': 72}


In [24]:
from langchain_community.document_loaders import WebBaseLoader
nike_job_url = "https://jobs.nike.com/job/R-39452"

loader = WebBaseLoader(nike_job_url)
page_data=loader.load().pop().page_content
print(page_data)

Apply for Software Engineer III

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer IIIBeaverton, OregonBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does more 

In [25]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
  """
    ###SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ###INSTRUCTION:
    The scraped text is from the career's page of a webiste.
    Your job is to extract the job postings and return them in JSON format 
    containing the following keys:
    `role`,`experience`,`skills` and `description`.
    Only reutn the valid JSON.
    ###VALID JSON( NO PREAMBLE): 
"""

)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Software Engineer III",
  "experience": "5+ years",
  "skills": [
    "ReactJs",
    "JavaScript/Node",
    "Microservice architecture",
    "Domain driven design",
    "RESTful APIs",
    "Serverless architectures",
    "Data structures",
    "Algorithms",
    "Architecture patterns",
    "Modern build strategies",
    "Continuous integration",
    "Unit testing",
    "Static analysis",
    "Automated integration tests",
    "NoSQL",
    "Version control systems (Git)",
    "Issue tracking tools (JIRA)",
    "CD/CI and related tools (BMX Jenkins, QMA, etc.)"
  ],
  "description": "As part of this team, you will be collaborating with engineers to design, develop, and implement next generation Digital Product Creation capabilities and services. With the changes in retail, you will be working on enabling new features and delivering new tools and solutions. You will be building new features in a cloud-based architecture using object-oriented programming systems/databa

In [26]:
from langchain_core.output_parsers import JsonOutputParser
jsonOP = JsonOutputParser()
json_res = jsonOP.parse(res.content)
type(json_res)

dict

In [27]:
import pandas as pd
df = pd.read_csv('my_portfolio.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [28]:
import chromadb, uuid

client = chromadb.PersistentClient('vectordb')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links":row["Links"]},
                       ids=[str(uuid.uuid4())]
                       )


In [29]:
links = collection.query(query_texts=["Experience in Python","Expertise in React Native"],n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [30]:
job = json_res
job['skills']

['ReactJs',
 'JavaScript/Node',
 'Microservice architecture',
 'Domain driven design',
 'RESTful APIs',
 'Serverless architectures',
 'Data structures',
 'Algorithms',
 'Architecture patterns',
 'Modern build strategies',
 'Continuous integration',
 'Unit testing',
 'Static analysis',
 'Automated integration tests',
 'NoSQL',
 'Version control systems (Git)',
 'Issue tracking tools (JIRA)',
 'CD/CI and related tools (BMX Jenkins, QMA, etc.)']

In [31]:


prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    ### INSTRUCTION:
    You are Huy, a Software Engineer, currently studying at VKU. 
    Over our experience, we have made numerous projects. Your job is to write a cold email to the client regarding the job mentioned above describing capability of VKU in making good students.
     Also add the most relevant ones from the following links to showcase Huy's portfolio: {link_list}
        Remember you are Huy, SE and currently studying at VKU. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job),"link_list":links})


print(res.content)

Subject: Expressed Interest in Software Engineer III Position and Showcasing VKU's Capabilities

Dear Hiring Manager,

I am Huy, a Software Engineer and a student at VKU, and I am excited to express my interest in the Software Engineer III position at your esteemed organization. As a student at VKU, I have had the privilege of being part of a community that fosters innovation, creativity, and excellence in the field of software engineering.

Throughout my academic journey, I have been exposed to a wide range of technologies and tools that have equipped me with the skills and knowledge required to excel in this role. VKU's rigorous curriculum and hands-on approach to learning have enabled me to develop a strong foundation in software engineering principles, including data structures, algorithms, and architecture patterns.

As a testament to VKU's capabilities in producing well-rounded software engineers, I would like to showcase some of my personal projects that demonstrate my skills an